In [17]:
import pandas as pd
import math
import sys
# !{sys.executable} -m pip install xlsxwriter
import xlsxwriter

In [18]:
df = pd.read_excel('1997-2020年鉴县级Ferda0920.xlsx',sheet_name="Sheet1")

In [19]:
df

,District,Province of City,City GDP (B/RMB),First Industry GDP (B/RMB),Population (0000),Rural Average Disposable Income (Yuan/Year),Second Industry GDP (B/RMB),Third Industry GDP (B/RMB),Total Retail Sales of Social Consumer Goods (B/RMB),Urban Average Disposable Income (Yuan/Year),...,Unnamed: 11,District.1,GDP(亿元),First Industry GDP (B/RMB).1,人口,农村居民人均可支配收入(元/人),Second Industry GDP (B/RMB).1,Third Industry GDP (B/RMB).1,社会消费品零售总额(亿元),城镇居民人均可支配收入(元/人)
0,Aba County,SiChuan,18.905400,0.0000,8.04670,15332.000,0.000000,0.0000,6.52305,37837.000,...,NaN,Aba County,18.9054,NaN,8,15332,NaN,NaN,6.52305,37837
1,Abaga Banner,Inner Mongolia,36.017700,10.4574,4.31180,30857.000,11.309100,14.2512,5.14400,40826.000,...,NaN,Abaga Banner,36.0177,NaN,0,30857,NaN,NaN,5.144,40826
2,A'ershan City,Inner Mongolia,19.720000,4.3200,4.33800,12612.000,2.640000,12.7600,4.91390,30987.000,...,NaN,A'ershan City,19.72,NaN,0,12612,NaN,NaN,4.9139,30987
3,Aheqi County,XinJiang,15.625900,0.0000,4.60120,0.000,0.000000,0.0000,5.90875,32158.000,...,NaN,Aheqi County,15.6259,NaN,4.6012,NaN,NaN,NaN,5.90875,32158
4,Aketao County,XinJiang,47.828400,0.0000,23.07460,8216.000,0.000000,0.0000,7.23258,32478.000,...,NaN,Aketao County,47.8284,NaN,23.0746,8216,NaN,NaN,7.23258,32478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2186,Zunyi Prefecture City,GuiZhou,1191.062265,129.0900,236.29000,17236.000,368.474265,693.4980,0.00000,39429.000,...,NaN,Zunyi Prefecture City,1191,128.9853,230.2095,NaN,368.019,693.9957,590.5,NaN
2187,Zuogong County,TiBet,0.000000,0.0000,4.66080,0.000,0.000000,0.0000,0.00000,0.000,...,NaN,Zuogong County,14.974,NaN,5.1625,NaN,NaN,NaN,NaN,NaN
2188,Zuoquan County,ShanXi,57.806600,4.9147,14.43970,7770.924,27.191200,25.7007,14.89639,30036.451,...,NaN,Zuoquan County,57.8066,NaN,16.4206,7770.92,NaN,NaN,14.89639,30036.45
2189,Zuoshui County,ShaanXi,76.538100,7.7300,13.83746,10304.000,37.740000,31.0700,9.66847,27522.000,...,NaN,Zuoshui County,76.54,NaN,13.84,10304,NaN,NaN,9.66847,27522


In [35]:
district1 = df.keys()[0]
district2 = df.keys()[12]
province = df.keys()[1]
# Column 1 and 2 are data for comparison, they are of the same categories
columns1 = df.keys()[2:10]
columns2 = df.keys()[13:21]
len(columns1)

8

In [43]:
output_file = 'test.xlsx'
workbook = xlsxwriter.Workbook(output_file)
worksheet = workbook.add_worksheet('县级数据核对结果')

# Write the first row (i.e., categories)
first_row = 0
for i in range(len(columns1)+2):
    # if math.isnan(row[j])
    worksheet.write(first_row, i, df.keys()[i])

# format the color of interested cells
cell_format = workbook.add_format({'bold': True, 'font_color': 'red'})

n_column = len(columns1)
new_data1 = []
total_mistake = []
count = 0

for index, row in df.iterrows():
    # write district and province
    # use district2 becasuse some values of district1 is blank
    worksheet.write(1+index, 0, row[district2])
    
    if type(row[province]) == str:   
        worksheet.write(1+index, 1, row[province])
    # if province is blank, name it to 'Unavailable'
    else:
        worksheet.write(1+index, 1, 'Unavailable')
    
    # data1 and data2 stores data of the same city on columns 1 and 2, iterate on each row. 
    data1=[]
    data2=[]

    # Collect raw data
    for i in columns1:
        if math.isnan(row[i]):
            row[i] = 0
        data1.append(row[i])

    for j in columns2:
        if (row[j]=='无结果') or (math.isnan(row[j])):
            row[j] = 0
        data2.append(row[j])

    # update data1
    for i in range(n_column):

        #如果data1为0，data2不为0，那么更新data1=data2
        if (data1[i]==0) and (data2[i]!=0):
            data1[i] = data2[i]
            worksheet.write(1+index, 2+i, data1[i])

        elif (data1[i]!=0) and (data2[i]!=0):
            #如果data1和data2都不为0，并且差异超过30%；记录下来,更新data1=data2,并标红
            if abs(data1[i]-data2[i])/data1[i]>0.3:
                data1[i] = data2[i]
                count += 1
                worksheet.write(1+index, 2+i, data1[i],cell_format)

            #如果data1和和data2都不为0，并且差异不大，更新data1=data2,不标红
            else:
                data1[i] = data2[i]
                worksheet.write(1+index, 2+i, data1[i])
                
        #如果data1不为0，data2为0，用data1的数据
        else:
            worksheet.write(1+index, 2+i, data1[i])
workbook.close()
print('差异过大的个数：',count)

差异过大的个数： 236
